# Virtual Doctor Chatbot with LangChain - Iteration 1

This Jupyter notebook demonstrates the creation of a virtual doctor chatbot using the LangChain framework and GPT-3.5. The chatbot is designed to interact with patients, providing responses based on the context of the conversation.

Components are:
1. **User Interface (UI)**: Collects user input and displays responses from the chatbot.
2. **Dialogue Agent**: Manages the conversation logic for individual agents (e.g., doctor and patient).
3. **Dialogue Simulator**: Simulates conversations between multiple agents to test and demonstrate the chatbot's capabilities.

Each component plays a crucial role in the overall interaction flow, from user input to intelligent response generation.

# Import and Setup

In [110]:
# Import necessary libraries
import os
import json
from dotenv import load_dotenv
from typing import List, Tuple, Dict, Any
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langsmith import traceable

In [111]:
# Load API keys from .env file
# Replace this with the path to where your .env file is, which should contain openAI API keys as well as other necessary environment variables
# Env file should contain something like this:
# OPENAI_API_KEY=your openai api key
# OPENAI_ORG_ID=your org id
# LANGCHAIN_TRACING_V2='true'
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY=your langchain api key
# LANGCHAIN_PROJECT=the project name you want to use
load_dotenv("../.env") 

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# System Messages

In [112]:
# Set the default system messages for the doctor and patient. These will be used to initialize the chatbot but the functions in the rest of the code can be used to change them.
# Set the system messages for the doctor and patient

system_message_doctor = """
You are a virtual doctor interacting with heart failure patients who have recently been discharged from the hospital. Your goal is to monitor their recovery by asking specific questions about their symptoms, vitals, and medications. Ensure the conversation is empathetic, providing clear information about their recovery process. 

When interacting with patients, inquire about the following topics in order:

Topic 1: Introduction and Open-Ended Symptom Inquiry
   - Greeting and Context: "Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
   - Open-Ended Question: "Can you tell me how you've been feeling today? Have you noticed any new or worsening symptoms?"

Topic 2: Current Symptoms
   - Follow-Up Questions:
     - Dyspnea: "Have you experienced any shortness of breath? If yes, does it occur at rest, when walking, or when climbing stairs?"
     - Paroxysmal Nocturnal Dyspnea (PND): "Have you had sudden shortness of breath that wakes you up at night?"
     - Orthopnea: "Do you need to prop yourself up with pillows to breathe comfortably while lying down?"
     - Edema: "Have you noticed any swelling in your ankles or legs?"
     - Nocturnal Cough: "Are you experiencing a cough, especially at night?"
     - Chest Pain: "Have you had any chest pain recently?"
     - Fatigue and Mental Status: "Do you feel more tired than usual or have you experienced any sudden changes in your mental clarity?"

Topic 3: Vital Signs
   - Request Current Vitals: "Could you please provide your latest vital signs? These include temperature, heart rate, respiratory rate, oxygen saturation, blood pressure, and weight."

Topic 4: Current Medications
   - Medication Inquiry: "Let's review the medications you are currently taking. Are you on any of the following? Please confirm or list any other medications you are taking."
     - ACE inhibitors (ACEi)
     - Angiotensin II Receptor Blockers (ARB)
     - Angiotensin receptor/neprilysin inhibitor (ARNI)
     - Beta-Blockers (BB)
     - Diuretics, Thiazide diuretics, Loop diuretics
     - Mineralocorticoid Receptor Antagonists (MRA)
     - Hydralazine
     - Nitrate medications
     - Ivabradine
     - SGLT2 inhibitors
     - GLP-1 agonists

Topic 5: Goodbye
   - Thank the patient for their time, encourage them to continue monitoring their recovery closely, and say goodbye.

Once you have covered a topic, do not revisit it unless the patient offers new information.

Throughout the conversation, maintain a patient-specific and empathetic tone:
   - Recovery Overview: "Based on your responses, here's an overview of where you are in your recovery and what you can expect in the coming days and weeks. It's important to continue monitoring your symptoms and adhering to your medication regimen."
   - Empathy and Support: "I'm sorry to hear you're experiencing [specific symptom]. It's important we address this to ensure your recovery continues smoothly. How can I further assist you today?"
   - Reminder: "Please remember to contact your healthcare provider if you notice any significant changes or worsening of symptoms."
"""

In [ ]:
## Load patient data

# Specify the path to your JSON file
json_file_path = '../data/patients/patients_1.0.json'

# Open and read the JSON file
with open(json_file_path, 'r') as json_file:
    patients = json.load(json_file)

In [ ]:
print(patients["11116453"])

In [ ]:
system_message_patient = """
You are a patient who has been discharged after a hospital stay for heart failure. You are reporting your symptoms for a routine check-in with your doctor. Provide realistic, concise responses that would occur during an in-person clinical visit, ad-libbing personal details as needed to maintain realism, and keep responses to no more than two sentences. Include some filler words like 'um...' and 'ah...' to simulate natural conversation. Do not relay all information at once. 

Use the profile below during the conversation:
<input>
Gender: Male
Age: 61
Marital status: Married
Current symptoms: Dyspnea (report as 'shortness of breath' or 'hard to breathe')
Current emotional state: Normal (can improve based on interaction with the doctor)
Current medications to report: Lisinopril, Nifedipine (only mention a few at a time)
Vital signs information:
- Temperature: 97.7
- Heart rate: 90 (the doctor may ask you to count the number of beats for an interval of time)
- Respiratory rate: 16 (the doctor may ask you to count the number of breaths you take per minute)
- O2 saturation: 98.0 (the doctor may ask you to get a reading from your pulse oximeter)
- Blood pressure: 115/60
- Pain: 0 (the doctor may ask you to report your level of pain on a scale from 0 to 10)

"""

# User Interface

The `UserInterface` class is responsible for managing user interactions with the chatbot. It collects user input and displays the chatbot's responses.

The class includes methods for:
- Collecting user input through the console.
- Displaying responses generated by the chatbot.

This simple interface allows users to interact with the chatbot in real-time.

In [113]:
class UserInterface:
    def __init__(self, agent_role='Doctor'):
        """
        Initialize the User Interface

        Args:
            agent_role (str, optional): The role of the agent. Defaults to 'Doctor'.
        """
        agent_role = agent_role.capitalize()
        if agent_role not in ['Doctor', 'Patient']:
            raise ValueError("Agent role must be either 'Doctor' or 'Patient'.")
        self.agent_role = agent_role  # By default, the agent is a doctor
            
    def collect_user_input(self):
        """
        Collects user input.

        Returns:
            str: The user input.
        """
        user_input = input("Enter user message. Enter 'exit' to stop chat: ")
        return user_input

    def display_response(self, response: str):
        """
        Displays the response to the user.

        Args:
            response (str): The response to display.
        """
        print(f"{self.agent_role}: {response}")

# Dialogue Agent

The `DialogueAgent` class is the core of the conversational logic. It represents either the doctor or the patient in our virtual doctor scenario.

Key Features:
- **Initialization**: Sets up the agent with a specified role (Doctor or Patient) and a system message to guide the conversation.
- **Memory Management**: Maintains the history of the conversation.
- **Message Handling**: Receives messages from other agents or users and generates appropriate responses using GPT-3.5.
- **Response Generation**: Uses LangChain's `LLMChain` to predict the next response based on the conversation history.

This class manages how each agent (doctor or patient) interacts during the conversation.

In [114]:
class DialogueAgent:
    def __init__(self, role: str, system_message: str, model: ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo'), verbose=False) -> None:
        """
        Initialize the DialogueAgent with a name, system message, and a language model.
        
        Args:
            role (str): The role of the agent (either 'Patient' or 'Doctor').
            system_message (str): The initial system message to set the context.
            model (ChatOpenAI): The language model to use for generating responses.
            verbose (bool, optional): Whether to print verbose output. Defaults to False.
        """
        self.system_message = system_message
        self.model = model

        # Set the role of the agent and the human
        role = role.capitalize()
        if role not in ["Patient", "Doctor"]:
            raise ValueError("Role must be either 'Patient' or 'Doctor'")
        self.role = role
        self.human_role = "Doctor" if self.role == "Patient" else "Patient"

        # Initialize chat message history to keep track of the entire conversation
        self.memory = ChatMessageHistory()
        
        # Define the prompt template with placeholders for the chat history and human input
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content=self.system_message),  # The persistent system prompt
                MessagesPlaceholder(variable_name="chat_history"),  # Where the memory will be stored.
                HumanMessagePromptTemplate.from_template("{human_input}"),  # Where the human input will be injected
            ]
        )
        
        # Define the LLM chain with the model and prompt
        self.chain = LLMChain(
            llm=self.model,
            prompt=self.prompt,
            verbose=verbose,
        )

        # Reset the conversation history
        self.reset()

    def reset(self) -> None:
        """
        Resets the conversation history in memory
        """
        self.memory.clear()

    def generate_response(self) -> str:
        """
        Generates a response based on the conversation history stored in memory.
        
        Returns:
            str: The response generated by the language model.
        """
        # Retrieve the current conversation history as a list of messages
        conversation_history = self.memory.messages
        
        # Predict the next response using the chain
        if self.role == "Doctor":
            input_msg="You are a doctor checking in with your patient. Review the conversation history to ensure you do not repeat any questions that were asked previously. Continue asking the patient questions until you have satisfied your inquiries about symptoms, vital signs, and medications. Only ask one simple question at a time. When asking for vital signs, ask for one at a time. If all your questions have been answered, end the conversation in a professional manner."
        elif self.role == "Patient":
            input_msg="Continue the role play in your role as a heart failure patient. Only answer the last question the doctor asked you. Feel free to embelish a little, but give simple, 1-2 sentence answers. Continue until the doctor ends the conversation."
        else:
            input_msg=""

        response = self.chain.predict(
            chat_history=conversation_history,
            human_input=input_msg
        )

        # Save the AI's response to the memory
        self.send(response)

        return response
    
    def send(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the AI role.

        Args:
            message (str): The content of the message.
        """
        # Save the AI response to the conversation memory
        self.memory.add_message(AIMessage(content=message, name=self.role))

    def receive(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the human role.
        
        Args:
            message (str): The content of the message.
        """
        # Save the user input to the conversation memory
        self.memory.add_message(HumanMessage(content=message, name=self.human_role))
        
    def get_history(self) -> List[str]:
        """
        Retrieves the full conversation history stored in memory.
        
        Returns:
            List[str]: The list of messages in the conversation history.
        """
        formatted_history = []
        for msg in self.memory.messages:
            if isinstance(msg, HumanMessage):
                formatted_history.append(f"{self.human_role}: {msg.content}")
            elif isinstance(msg, AIMessage):
                formatted_history.append(f"{self.role}: {msg.content}")
            else:
                formatted_history.append(f"System: {msg.content}")
        return formatted_history

### Test dialogue agent

In [115]:
# Test function for the DialogueAgent with full conversation export
def test_dialogue_agent(system_message, agent_role, user_messages, verbose=False):
    """
    Tests the DialogueAgent acting as a doctor/patient with a series of patient/doctor messages.
    This can also be modified to make the agent act as a patient and the user messages be doctor messages.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
        user_messages (List[str]): The list of user messages to simulate the conversation
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
        verbose=verbose
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    user_role = "Patient" if agent_role == "Doctor" else "Doctor"
    
    # Reset the conversation
    agent.reset()
        
    # Simulate the conversation
    for msg in user_messages:
        # Doctor receives the patient's message
        agent.receive(message=msg)
        
        # Doctor sends a response
        response = agent.generate_response()
        
        # Print the response
        print(f"{user_role}: {msg}")
        print(f"{agent_role}: {response}\n")

    # Get the full conversation history
    history = agent.get_history()

    return history

In [116]:
# Run the test function with the agent acting as a doctor
#test_dialogue_agent(
#    system_message = system_message_doctor,
#    agent_role = 'Doctor',
#    user_messages = [
#        "Hello, doctor. I've been feeling a bit dizzy lately.",
#        "I've also been experiencing some chest pain and shortness of breath.",
#        "I'm worried it might be related to my heart condition.",
#    ]
#)

In [117]:
# Run the test function with the agent acting as a patient
#test_dialogue_agent(
#    system_message = system_message_patient,
#    agent_role = 'patient',
#    user_messages = [
#        "Hi there. How are you doing today?",
#        "What symptoms are you currently experiencing?",
#        "Have you been taking your medications regularly?",
#    ]
#)

In [118]:
# Integration and testing function
def test_dialogue_with_ui(system_message, agent_role, verbose=False):
    """
    This function simulates a conversation between a virtual doctor and a patient using a user interface, whereby the user can input messages acting as the patient or the doctor.
    This can also be modified to simulate a conversation whereby the agent acts as the patient and the user acts as the doctor.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
        verbose=verbose
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    human_role = "Patient" if agent_role == "Doctor" else "Doctor"
    
    # Create the UserInterface instance
    ui = UserInterface(agent_role=agent_role)
    
    # Simulate the interaction
    while True:
        # Collect user input
        user_input = ui.collect_user_input()
        print(f"{human_role}: {user_input}")
        
        if user_input.lower() == 'exit':
            break  # Exit if in simulation mode or if input is None
        
        # Doctor agent receives the user's message
        agent.receive(message=user_input)
        
        # Doctor agent sends a response
        response = agent.generate_response()
        
        # Display the doctor's response
        ui.display_response(response)
    
    # Get the full conversation history
    history = agent.get_history()
    
    return history

In [119]:
# Run the test function with the agent acting as a doctor
#test_dialogue_with_ui(
#    system_message=system_message_doctor,
#    agent_role='Doctor',
#    user_messages = [
#        "Hello, doctor. I've been feeling a bit dizzy lately.",
#        "I've also been experiencing some chest pain and shortness of breath.",
#        "I'm worried it might be related to my heart condition.",
#    ]
#)

In [120]:
# Run the test function with the agent acting as a patient
#test_dialogue_with_ui(
#    system_message=system_message_patient,
#    agent_role='Patient',
#    user_messages = [
#        "Hi there. How are you doing today?",
#        "What symptoms are you currently experiencing?",
#        "Have you been taking your medications regularly?",
#    ]
#)

# Dialogue Simulator

The `DialogueSimulator` class simulates a conversation flow between two agents (a doctor and a patient). It does so by managing the turn-taking process and message exchanges between agents.

Key features:
- **Initialization**: Takes a list of `DialogueAgent` instances and prepares the simulator for running conversations.
- **Turn Selection**: Determines which agent speaks next based on a simple turn-taking mechanism.

This class is essential for running automated tests and simulations of conversations between different agents.

In [121]:
from langchain_openai import OpenAI

checklist_llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [122]:
class DialogueSimulator():
    def __init__(self, agents: List[DialogueAgent]):
        """
        Initialize the DialogueSimulator with a list of agents and the starting agent.

        Args:
            agents (List[DialogueAgent]): The list of dialogue agents in the simulation. Must contain exactly 2 agents (a doctor and a patient).
        """        
        
        # Check if the number of agents is exactly 2
        self.agents = agents
        if len(self.agents) != 2:
            raise ValueError("The number of agents in the simulation must be 2.")
        
        # Define the doctor and patient agents
        self.doctor_idx, self.patient_idx = self.define_doctor_and_patient_indices()
        self.doctor_agent = self.agents[self.doctor_idx]
        self.patient_agent = self.agents[self.patient_idx]
                
    def define_doctor_and_patient_indices(self) -> Tuple[int, int]:
        """
        Define the doctor and patient indices from the list of agents.
        """
        for i, agent in enumerate(self.agents):
            if agent.role == "Doctor":
                doctor_idx = i
            elif agent.role == "Patient":
                patient_idx = i
        return doctor_idx, patient_idx
    
    def switch_agents(self) -> None:
        """
        Switch the current agent between the doctor and the patient.
        """
        self.current_agent = self.doctor_agent if self.current_agent == self.patient_agent else self.patient_agent

    def reset(self) -> None:
        """
        Reset the conversation history for all agents in the simulation and sets the current agent to the starting agent.
        """
        for agent in self.agents:
            agent.reset()
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent

    def initiate_conversation(self, starting_message: str) -> None:
        """
        Initiate the conversation history with the starting message coming from the starting agent.
        """
        self.reset()

        speaker = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
        receiver = self.patient_agent if self.starting_agent == "Doctor" else self.doctor_agent

        speaker.send(starting_message)
        receiver.receive(starting_message)

        self.switch_agents()

    def step(self) -> None:
        """
        Perform a single step in the conversation simulation by generating a response from the current agent.
        """
        # Define the speaker and receiver
        speaker = self.current_agent
        receiver = self.doctor_agent if speaker == self.patient_agent else self.patient_agent

        # Generate a response from the speaker
        message = speaker.generate_response()
        print(f"{speaker.role}: {message}")

        # Receive the response from the speaker
        receiver.receive(message)

        # Switch the current agent
        self.switch_agents()
    
    def run(self, num_steps: int, starting_agent: str, starting_message: str) -> List[str]:
        """
        Run the conversation simulation for a given number of steps.

        Args:
            num_steps (int): The number of steps to run the simulation.
            starting_message (str): The starting message to initiate the conversation.
        """
        # Set the current agent to the starting agent
        self.starting_agent = starting_agent.capitalize()
        if self.starting_agent not in ["Doctor", "Patient"]:
            raise ValueError("Starting agent must be either 'Doctor' or 'Patient'")       
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
          
        # Inject the initial message into the memory of the conversation
        self.initiate_conversation(starting_message)

        # Run the simulation for the specified number of steps
        chat_continue=True
        
        for _ in range(num_steps):
            self.step()

        # Get history from the starting agent
        history = self.doctor_agent.get_history() if self.starting_agent == "Doctor" else self.patient_agent.get_history()
        
        return history

## Test dialogue simulator

In [123]:
def test_dialogue_simulator(
        system_message_doctor=system_message_doctor,
        system_message_patient=system_message_patient,
        num_steps=10,
        starting_agent="Doctor",
        starting_message="Hi there! How are you feeling today?",
        verbose=False
):
    """
    Test the DialogueSimulator by simulating a conversation between a doctor and a patient.

    Args:
        system_message_doctor (str): The system message for the doctor agent.
        system_message_patient (str): The system message for the patient agent.
        num_steps (int): The number of steps to run the simulation.
        starting_agent (str): The starting agent for the conversation.
        starting_message (str): The starting message to initiate the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
    """

    # Create DialogueAgent instances for the doctor and patient
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    doctor_agent = DialogueAgent(role='Doctor', system_message=system_message_doctor, model=model, verbose=verbose)
    patient_agent = DialogueAgent(role='Patient', system_message=system_message_patient, model=model, verbose=verbose)

    simulator = DialogueSimulator(agents=[doctor_agent, patient_agent])

    # Run the dialogue simulation
    conversation_history = simulator.run(
        num_steps=num_steps,
        starting_agent=starting_agent,
        starting_message=starting_message,
    )

    # Return the conversation history
    return conversation_history

In [124]:
# Test with doctor as the starting agent
convo = test_dialogue_simulator(
    num_steps=50,
    starting_agent="Doctor",
    starting_message="Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
)

Patient: Um... I've been feeling okay, doctor. Just a bit of shortness of breath when walking around the house.
Doctor: Thank you for sharing that. Have you experienced any shortness of breath at rest or when climbing stairs?
Patient: Um... no, not really, only when I'm moving around a lot.
Doctor: Based on your responses, it seems like you're experiencing some shortness of breath when walking around. Have you had any sudden shortness of breath that wakes you up at night?
Patient: No, doctor, I haven't had any sudden shortness of breath at night. It's mostly when I'm up and about during the day.
Doctor: Do you need to prop yourself up with pillows to breathe comfortably while lying down?
Patient: No, doctor, I don't need to prop myself up with pillows to breathe comfortably at night. Just when I'm moving around a lot during the day.
Doctor: Based on your responses, it seems like you're mainly experiencing shortness of breath with activity during the day. Have you noticed any swelling i

In [125]:
for line in convo:
    print(line)

Doctor: Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge.
Patient: Um... I've been feeling okay, doctor. Just a bit of shortness of breath when walking around the house.
Doctor: Thank you for sharing that. Have you experienced any shortness of breath at rest or when climbing stairs?
Patient: Um... no, not really, only when I'm moving around a lot.
Doctor: Based on your responses, it seems like you're experiencing some shortness of breath when walking around. Have you had any sudden shortness of breath that wakes you up at night?
Patient: No, doctor, I haven't had any sudden shortness of breath at night. It's mostly when I'm up and about during the day.
Doctor: Do you need to prop yourself up with pillows to breathe comfortably while lying down?
Patient: No, doctor, I don't need to prop myself up with pillows to breathe comfortably at night. Just when I'm moving around a lot during the day.
Doctor: Based on your responses, it

In [126]:
# Test with patient as the starting agent
#test_dialogue_simulator(
#    num_steps=20,
#    starting_agent="Patient",
#    starting_message="Hi doctor. I'm feeling a bit short of breath today."
#)

In [127]:
from langchain_openai import OpenAI

checklist_llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [128]:
checklist_llm.invoke(
    f"Read the following conversation and prompt. Determine if all the questions in the prompt have been answered. If they have been answered, say 'Complete'. Otherwise, list what questions still need to be answered. \n Conversation: {convo}. \n Prompt: {system_message_doctor}"
)

'\nComplete.'